This is the large casestudy for the damage dataset. Data that we are reading is 34Gb and device that I have is 32 GB so technically I cannot play around with this data

In [3]:
from dask.distributed import Client, progress
import tables
import numpy as np
Client()

Client Scheduler: tcp://127.0.0.1:44557 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.59 GB


In [4]:
import dask.array as da

In [5]:
path="/home/berkay/Projects/pymks/data/ms_copy_data.hdf5"


In [6]:
tps_hdf5_file = tables.open_file(path, mode='r')


In [7]:
ms_data=tps_hdf5_file.root.ms_data

# Data Construction
Creating the Large Dask Array From HDF file:<br/>
Target Data is n* 51* 51*51<br/>
So I will first read as NP array then will construct the dask array. Basically , we are stitching them together.

In [8]:
chunksize=1300
dask_arrays = []
for i in range(5):
#     ms_chunk=np.array(ms_data[i*chunksize:(i+1)*chunksize])
#     ms_chunk=ms_chunk[:,0:51**3]
#     ms_chunk=ms_chunk.reshape((chunksize,51,51,51))
    ms_chunk=da.from_array(np.array(ms_data[i*chunksize:(i+1)*chunksize,0:51**3]).reshape(chunksize,51,51,51),chunks=(chunksize,51,51,51))
    print("%d Chunks completed"%(i+1))
    dask_arrays.append(ms_chunk)

1 Chunks completed
2 Chunks completed
3 Chunks completed
4 Chunks completed
5 Chunks completed


In [9]:
# ms_data.shape[0]//chunksize
x = da.concatenate(dask_arrays, axis=0)  # concatenate arrays along first axis

In [10]:
x

dask.array<concatenate, shape=(6500, 51, 51, 51), dtype=float32, chunksize=(1300, 51, 51, 51), chunktype=numpy.ndarray>

In [23]:
type(x)

dask.array.core.Array

In [24]:
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation,FlattenTransformer
from dask_ml.decomposition import PCA
from sklearn.pipeline import Pipeline

In [25]:
bl = Pipeline(steps=[
    ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]))])

# There is an issue 

In [26]:
bl.transform(x)

dask.array<stack, shape=(6500, 31, 31, 31, 1), dtype=float64, chunksize=(6500, 31, 31, 31, 1), chunktype=numpy.ndarray>